In [1]:
###########################################
###########################################
#     WORKSPACE TO MESS WITH AI VJ Data
###########################################
###########################################

import sys
import spotipy
import spotipy.util as util


import calendar
import dateutil
import json
import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from numpy import nan as Nan

import datetime

from utils import *
#from spotify_utils import *

#scope = 'user-library-read'
#scope = 'user-top-read user-read-playback-state'
scope = 'user-read-playback-state user-read-recently-played'

#token = util.prompt_for_user_token('aaronopp', scope, client_id='dbe2a20785304190b8e35d5d6644397b', client_secret='cc259d8378be48beaad9171a5afb19ba', redirect_uri='http://localhost:8888/callback')
import dateutil.parser as dp


SyntaxError: invalid syntax (utils.py, line 863)

In [2]:
from server import block_until_token

import webbrowser

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
scope = 'user-read-playback-state user-read-recently-played'

#############################
# Initalize spotify web API
#############################

print '\n trying to create spotify data and marry it \n'
username = sys.argv[1]

print "GETTING TOKEN"
sp_oauth = spotipy.oauth2.SpotifyOAuth(client_id='dbe2a20785304190b8e35d5d6644397b', 
    client_secret='d73cf4a1525c44e899feeeff4b840040', redirect_uri='http://localhost:5555/redirect',  scope=scope)
auth_url = sp_oauth.get_authorize_url()

webbrowser.open(auth_url)
token = block_until_token(sp_oauth)



 * Running on http://127.0.0.1:5555/ (Press CTRL+C to quit)



 trying to create spotify data and marry it 

GETTING TOKEN


127.0.0.1 - - [27/Mar/2018 12:20:27] "GET /redirect?code=AQAERU4Y5Je_QQJSoYgxs10PaPyl216c65ovsf9JSUGb2PzdZcqk-iWssXM_a1kVzGSxIDqRCbIaP1iHU6ycIW2XUn5jBEBEcoQ7Y3v5aO1JJlyxxRtaqDfcMqHAX-A2QILuZndkEmIGnw4QuOGHS0WJapFrCzYI5CKDbWKvbxHMMIVUcpgSkcappl6dreAPrZPs3DOonIN7jjfc3WwDjbcaK_erfMdIBqnr2bf_uIIvfw00vohAxnRdx_VvAdnDbn7iLX35OB83 HTTP/1.1" 200 -


In [3]:
token

u'BQBWOSIOkJgiBUnKHgQGq01rQ-TPYZ1CV97rfy3YXIOl6zE9AGXdYcXwsO44NiKHjERd0eqEfjSny1N3SjqJBbWPyLku3cwQoNFs4ih9QDc3OovTihoie9312YVLXv1PT7GAl4-mIyBlCalOVPFJHJQTH4uy'

In [74]:
import sys

import spotipy
import spotipy.util as util


import calendar
import dateutil
import json
import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

#scope = 'user-library-read'
#scope = 'user-top-read user-read-playback-state'
# scope = 'user-read-playback-state user-read-recently-played'

# token = util.prompt_for_user_token('aaronopp', scope, client_id='dbe2a20785304190b8e35d5d6644397b', client_secret='cc259d8378be48beaad9171a5afb19ba', redirect_uri='http://localhost:8888/callback')
# import dateutil.parser as dp



# just realized i should only create/save features at the end when i have all ids. then i
# only execute it when i need it!
def spotify_pipeline(token, scope, time_test_scaled, filename, save=True):
    
    #if not os.path.exists(filename):
        #print 'data folder created'
        #os.makedirs(filename)

    spotify_data_df = create_spotify_df(token, scope, time_test_scaled)
    if save:
        spotify_data_df.to_csv(filename + '_raw.csv')
    print spotify_data_df.index[1]

    print time_test_scaled[1:4]
    print spotify_data_df.index[1]- time_test_scaled[1]
    spotify_dataset = create_spotify_dataset(spotify_data_df, time_test_scaled)
    if save:
        spotify_dataset.to_csv(filename + '.csv')
    return spotify_data_df

def save_spotify_df(token, scope, filename):
	
    spotify_data_df = create_spotify_df(token, scope)
    spotify_data_df.to_csv(filename)

def create_spotify_df(token, scope, get_genre=True, save_features=False):
    timestamp = []
    songs = []
    artists = []
    popularities = []
    ids = []
    artist_ids = []
    genres = []
    durations = []

    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace=False
        ranges = ['short_term', 'medium_term', 'long_term']
        #esults = sp.current_user_playing_track()
        curr_results = sp.current_playback()
        #print 'current playback keys ', curr_results.keys()
        #print results['item']
        #print 'user playing track keys', results.keys()
        if curr_results != None:
            
            print 'song name: ', curr_results['item']['name']
            print 'timestamp: ', curr_results['timestamp']
            curr_artist_info = curr_results['item']['artists']
            print 'artist name: ', curr_artist_info[0]['name']
            print 'artist id: ', curr_artist_info[0]['id']



            #print curr_results['timestamp']
            #print 'type name: ', curr_results['item']['type']
            print 'song id: ', curr_results['item']['id']
            
            print '\n starting spotify data with current playing song! \n'

            timestamp.append(int(str(curr_results['timestamp']).encode('utf-8')))
            songs.append(curr_results['item']['name'].encode('utf-8'))
            ids.append(curr_results['item']['id'].encode('utf-8'))

            artists.append(curr_artist_info[0]['name'].encode('utf-8'))
            artist_ids.append(curr_artist_info[0]['id'].encode('utf-8'))

            popularities.append(curr_results['item']['popularity'])
            #if get_genre == True:
            genres.append(get_genre_from_track_id(sp, curr_results['item']['id'].encode('utf-8')))
            durations.append(curr_results['progress_ms'])
        else: 
            print '\n no track currently playing \n'
                    
        recent_results = sp.current_user_recently_played(limit=5)
        #print recent_results.keys()

        #print recent_results['href']
        #print recent_results['cursors']
        #print recent_results['next']

        for i, item in enumerate(recent_results['items']):
            print 'name: ', item['track']['name']
            print 'id: ', item['track']['id']
            artist_info = item['track']['artists']
            print 'artist: ', artist_info[0]['name']
            print 'duration ms: ', item['track']['duration_ms']

            print 'popularity: ', item['track']['popularity']


            print 'played: ', item['played_at']
            print 'type', type(item['played_at'])
            timestamp.append(item['played_at'].encode('utf-8'))
            songs.append(item['track']['name'].encode('utf-8'))
            ids.append(item['track']['id'].encode('utf-8'))

            artists.append(artist_info[0]['name'].encode('utf-8'))
            artist_ids.append(artist_info[0]['id'].encode('utf-8'))

            popularities.append(item['track']['popularity'])
            #if get_genre == True:
            genres.append(get_genre_from_track_id(sp, item['track']['id'].encode('utf-8')))
            durations.append(item['track']['duration_ms'])

        #get timestamps
        utc_timestamps = []
        for index, time in enumerate(timestamp):
            if index != 0:
                utc_timestamps.append(calendar.timegm(dateutil.parser.parse(time).timetuple())*1000)
            else:
                utc_timestamps.append(time)
        print utc_timestamps
        # create dataframe of basic data.
        spotify_data = {'timestamp': utc_timestamps, 'artist': artists, 'artist_id': artist_ids, 'song': songs, 'id': ids, 'genre': genres, 'popularity': popularities, 'duration': durations}
        
        spotify_data_df = pd.DataFrame(spotify_data)
        spotify_data_df = spotify_data_df.set_index('timestamp')
        
        if save_features == True:
            try: 
                save_spotify_features(sp, ids)
                print 'features saved!'
            except:
                print 'unable to save features'

    return spotify_data_df

def save_spotify_features(sp, ids):
    # get audio features for each track and save as a JSON!
    features = sp.audio_features(ids)
    features_json = []
    for feature in features:
        #print feature
        print(json.dumps(feature, indent=4))
        print()
        features_json.append(json.dumps(feature, indent=4))
        with open('data.json', 'a') as outfile:
            json.dump(feature, outfile, indent=4)
        #analysis = sp._get(feature['analysis_url'])
        #print(json.dumps(analysis, indent=4))
        #print()
def get_album_id_from_track(sp, id):
    track = sp.track(id)
    #pprint.pprint(track)
    album_id = track['album']['id']
    
    print album_id
    return album_id
def get_genre_from_track_id(sp, id):
    track = sp.track(id)
    #pprint.pprint(track)
    album_id = track['album']['id']
    album = sp.album(str(album_id))
    pprint.pprint(album['genres'])
    return album['genres']

def get_audio_features_json(sp, dataframe, savefile):
    # get audio features for each track and save as a JSON!
    ids = spotify_data['id'].tolist()
    features = sp.audio_features(ids)
    features_json = []
    for feature in features:
        #print feature
        print(json.dumps(feature, indent=4))
        print()
        features_json.append(json.dumps(feature, indent=4))
        with open(savefile, 'a') as outfile:
            json.dump(feature, outfile, indent=4)
        # OPTIONAL song analysis (too much irrelavant data ATM.)    
        
        #analysis = sp._get(feature['analysis_url'])
        #print(json.dumps(analysis, indent=4))
        #print()

####################################################
#  Functions to match song data with timestamp data
#
####################################################
def match_spotify_dataset(dataframe, time_test_scaled):
    j = 0
    Y_logger_df = pd.DataFrame(data=None, columns=dataframe.columns)

    for index_logger, timestamps in enumerate(time_test_scaled):
        for index, value in enumerate(list(dataframe.index)):
            print '\n \n value: ', value 
            if value < timestamps:
                print value - timestamps
                #print dataframe['duration'].iloc(index)
                #print type(dataframe['duration'].iloc(index))
                print dataframe.iloc[0]['duration']
                #if timestamps - value > dataframe.iloc[0]['duration']:
                    #print '\n \n \n song is not actually playing'
                    #spotify_series = pd.Series([Nan, Nan, Nan, Nan, Nan, Nan, Nan], 
                    #                           index=['artist', 'artist_id', 'duration', 'genre', 'id',  
                    #                                'popularity', 'song'], name=value)
                    #Y_logger_df = Y_logger_df.append(spotify_series)
                    #break
                    
                print index
                print dataframe.iloc[index]
                Y_logger_df = Y_logger_df.append(dataframe.iloc[index])
                j += 1
                break
    print 'j:' , j
    if j == 0:
        print '\n \n NO MATCHES - please extend your recent playback! \n \n'
        print 'terminating script'
        sys.exit(1)
    return Y_logger_df

def create_spotify_dataset(dataframe, time_test_scaled, save_df=False):
    y_df = match_spotify_dataset(dataframe, time_test_scaled)
    y_df['timestamps'] = time_test_scaled
    y_df = y_df.reset_index()
    y_df = y_df.rename(columns={'index': 'song_start_ts'})
    y_df = y_df.set_index('timestamps')
    if save_df == True:
        y_df.to_csv('spotify_dataset.csv')
    return y_df

# to write- get all IDs from big pandas dataframe and then save spotify features
# save spotify data df as dataframe? or just sync it w training data.

In [49]:
type(spotify_df_raw.iloc[0]['duration'])

numpy.int64

In [31]:
now = datetime.datetime.now()
time_tag = str(now.hour) + '_' + str(now.minute)

    
AI_VJ_FOLDER = '/Users/aaronopp/Desktop/SymmetryLabs/winter_sun/SLStudio/AI_VJ/'
WEIGHTS_FOLDER = AI_VJ_FOLDER + 'model_weights/v2/'
TRAINING_DATA_FOLDER = AI_VJ_FOLDER + 'training_data/'
PROCESSING_OUTPUT = AI_VJ_FOLDER + 'logger/data/out.txt'
DATA_FOLDER = TRAINING_DATA_FOLDER + 'notebooks' + '/'+ str(now.month) + '_'+ str(now.day) + '/'


In [32]:
######################################################################
#                      GET data FROM LOGGER
######################################################################


#data_file = "/Users/aaronopp/Desktop/logger/data/out.txt"
TRAINING_DATA_PATH = '/Users/aaronopp/Desktop/SymmetryLabs/winter_sun/SLStudio/AI_VJ/training_data/aaronopp/2_2/'
timestring = '11_49'
data_file =  TRAINING_DATA_PATH + 'logger_training_output_raw_' + timestring + '.txt'
%pwd
X_test = np.load(TRAINING_DATA_PATH + 'x_train_raw_' + timestring + '.npy')
time_test_scaled = np.load(TRAINING_DATA_PATH + 'time_test_scaled_raw_' + timestring +'.npy')




In [6]:
if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace=False

    ranges = ['short_term', 'medium_term', 'long_term']
    #esults = sp.current_user_playing_track()
    curr_results = sp.current_playback()
    #save_spotify_df(token, scope, 'fullspotifytest')

    spotify_df_raw = spotify_pipeline(token, scope, time_test_scaled, DATA_FOLDER + 'spotify_data_' + time_tag, False)


NameError: name 'spotify_pipeline' is not defined

In [80]:
spotify_df_raw.iloc[0]['duration']
#dataframe.iloc[index]

79365

In [68]:
spotify_series = pd.Series([Nan, Nan, Nan, Nan, Nan, Nan, Nan], index=['artist', 'artist_id', 'duration', 'genre', 'id',  
                                                                      'popularity', 'song'], name=value)
#    s2 = pd.Series([Nan,Nan,Nan,Nan], index=['A', 'B', 'C', 'D'])
#>>> result = df1.append(s2)
#value = 10
#list2 = [value, Nan, Nan, Nan, Nan, Nan, Nan, Nan]
#spotify_series = pd.Series([value, Nan, Nan, Nan, Nan, Nan, Nan, Nan], index=['timestamps'], 
                     #      columns=['artist', 'artist_id', 'duration', 'genre', 'id',  
                      #                                               'popularity', 'song'])

In [76]:
spotify_df_raw#.append(spotify_series, ignore_index=False)

,artist,artist_id,duration,genre,id,popularity,song
timestamp,,,,,,,
1517602746693,The Jungle Giants,6wFwvxJkurQPU2UdeD4qVt,112953,[],7t8CoYp10pe19hTxsjnL51,34,Lemon Myrtle
1517597260000,Ugly Casanova,6LAwiATl5lQ3Uwh3llJpJI,298960,[],1qkPQySG2xFixO64S2Uxds,38,Hotcha Girls
1517597050000,Ugly Casanova,6LAwiATl5lQ3Uwh3llJpJI,209373,[],4HHo0uyqgBgoFVBKS7ux7M,39,Things I Don't Remember
1517596878000,Ugly Casanova,6LAwiATl5lQ3Uwh3llJpJI,171706,[],265Tszg4mDqj1b6I0xJrXJ,42,Here's To Now
1517596572000,Ugly Casanova,6LAwiATl5lQ3Uwh3llJpJI,305280,[],3Khizgd763J4s63Gcv6D9u,47,Barnacles
1517544463000,Queens of the Stone Age,4pejUc4iciQfgdX6OKulQn,320268,[],5NbMimOzRykGsE1tVHDtKV,65,Domesticated Animals


In [77]:
time_test_scaled

array([1517600693908, 1517600698947, 1517600703909, 1517600708916,
       1517600713923, 1517600718931, 1517600723936, 1517600728903,
       1517600733966, 1517600738996, 1517600744004, 1517600749012,
       1517600753980, 1517600758987, 1517600763994, 1517600769000,
       1517600774008, 1517600779014, 1517600783985, 1517600788992,
       1517600794003, 1517600799004, 1517600804011, 1517600808982,
       1517600813986, 1517600818994, 1517600824001, 1517600829007,
       1517600833982, 1517600838987, 1517600843992, 1517600848999,
       1517600854005, 1517600859061, 1517600864057, 1517600869061,
       1517600874074, 1517600879072, 1517600884082, 1517600889052,
       1517600894057, 1517600899066, 1517600904073, 1517600909080,
       1517600914050, 1517600919059, 1517600924065, 1517600929070,
       1517600934075, 1517600939088, 1517600944061, 1517600949064,
       1517600954067, 1517600959072, 1517600964080, 1517600969055,
       1517600974055])

In [69]:
print spotify_df_raw.index[2]-spotify_df_raw.index[1]
spotify_df_raw['duration'].iloc(2)

-59000


In [64]:
spotify_df_raw

,artist,artist_id,duration,genre,id,popularity,song
timestamp,,,,,,,
1517524316210,The Doobie Brothers,39T6qqI0jDtSWWioX8eGJz,79365,[],2D86Et1GcZEP2XITcmZTs9,19,Long Train Runnin'
1517524174000,Jack White,4FZ3j1oH43e7cukCALsCwf,219400,[],3T76zPJz3tWL27FrjJe2ot,62,Lazaretto
1517524115000,CHVRCHES,3CjlHNtplJyTf9npxaPl5w,258159,[],1dmeXS21t9WV6d3Fm5mxTi,48,Do I Wanna Know?
1517524054000,The Shins,4LG4Bs1Gadht7TCrMytQUO,284466,[],5rhQo26ggW0kpAieLLUWaq,52,Painting a Hole
1517524012000,The Shins,4LG4Bs1Gadht7TCrMytQUO,189493,[],6qnaCx4wQQBqFd9XdQyWjC,66,Name For You
1517523742000,David Bowie,0oSGxfWSnnOXhD2fKuz2Gy,254295,[],0pQskrTITgmCMyr85tb9qq,74,Starman - 2012 Remastered Version


In [4]:

## patterns 1_5
def create_Y_pattern(dataframe, channel, patterns_full):
    pattern_list = []
    route = '/lx/channel/' + str(channel) + '/activePattern' 
    if route in dataframe.columns:

        for i in dataframe[route]:
            pattern_list.append(i)
    else:
        print 'patterns not in dataframe, moving on'

    pattern_labels = get_unique_tags(pattern_list)
    Y_pattern = np.zeros((len(pattern_list), len(patterns_full)))
    
    for index, label in enumerate(pattern_list):
        #print type(i)
        #print 'ind' , index
        #print label
        Y_pattern[index] = encode_class(label, patterns_full)
        
    return Y_pattern
def create_Y_global(Y_logger_df):

######################################################################
#                   Raw values to labels
######################################################################
    
    print 'converting dataframe to labels'
    print 'dataframe length: ', len(Y_logger_df)
    print 'dataframe columns: ', Y_logger_df.columns
    
    color_labels = ['red', 'orange', 'yellow', 'green', 'teal', 'blue', 'purple', 'magenta', 'red']
    color_labels_encoding = ['red', 'orange', 'yellow', 'green', 'teal', 'blue', 'purple', 'magenta']
    speed_range_scaled = [0, 34, 67, 101]
    speed_labels = ['slow', 'medium', 'fast']




######################################################################
#         create individual lists and pattern label array
######################################################################
    color_list = []
    speed_list = []
    
    
    if '/lx/palette/color/hue' in Y_logger_df.columns:
        print 'hue is in dataframe, making labels'

        for i in Y_logger_df['/lx/palette/color/hue']:
            #print type(i)
            color_list.append(hue_to_color(float(i)))
    else:
        print 'hue is not in dataframe, moving on'

    if '/lx/engine/speed' in Y_logger_df.columns:

        for i in Y_logger_df['/lx/engine/speed']:
            speed_list.append(speed_to_label(float(i)))
    else:
        print 'speed not in dataframe, moving on'
        
 


    Y_color = np.zeros((len(color_list), len(color_labels_encoding)))
    Y_speed = np.zeros((len(speed_list), len(speed_labels)))
    
    
    #Y_pattern = np.zeros((len(pattern_list), len(pattern_labels)))

    for index, label in enumerate(color_list):
        #print type(i)
        #print 'ind' , index
        #print label
        Y_color[index] = encode_class(label, color_labels_encoding)

    for index, label in enumerate(speed_list):
        #print type(i)
        #print 'ind' , index
        #print label
        Y_speed[index] = encode_class(label, speed_labels)



    return Y_color, Y_speed
def create_Y_effect_by_channel(dataframe, effect_number, channel):
    
    global effect_labels_full
    Y_effect = [None] * len(dataframe)
    
    i = 0
    
    for index, row in dataframe.iterrows():
        #print 'row:', row['/lx/channel/1/effect/1/enabled'], 
        if row['/lx/channel/' + str(channel) + '/effect/' + str(effect_number) + '/enabled'] == '1':
            #print float(row['/lx/channel/1/effect/' + str(effect_number) + '/amount'])
            #print i
            if row['/lx/channel/' + str(channel) + '/effect/' + str(effect_number) + '/amount'] != '0':
                #print '0value although its on! - at row: ', i
                #continue
                Y_effect[i] = effect_to_label(float(row['/lx/channel/' + str(channel) + '/effect/' + str(effect_number) + '/amount']))
            else: 
                print '0 value although its on! - at row: ', i
            #print 'effect on at: ', row['/lx/channel/1/effect/' + str(effect_number) + '/amount']
        else:
            print 'blur off'
        i += 1    
    #for index, label in enumerate(Y_blur):
        #Y_pattern[index] = encode_class(label, patterns_full)

    Y_effect_encoded = np.zeros((len(Y_effect), len(effect_labels_full)))
    
    for index, label in enumerate(Y_effect):
        Y_effect_encoded[index] = encode_class(label, effect_labels_full)
    return Y_effect, Y_effect_encoded

def logger_to_training_data_all_channels(data_file):
    datan = pd.read_csv(data_file, sep="]", header=None, error_bad_lines=False)
    datan_2 = pd.DataFrame(datan[0].str.rsplit(',',1).tolist(), columns = ['lx_route','data'])
    datan_ts = pd.DataFrame(datan[1].str.rsplit(',',1).tolist(), columns = ['raw','timestamps'])
    timestamps2 = datan_ts['timestamps'].str.extract('(\d+)').fillna(0).astype(int)
    #datan_2.set_index(timestamps2)
    for col in datan_2:
        datan_2 = datan_2.replace('{"route":"', '', regex= True)
        datan_2 = datan_2.replace('"data":', '', regex= True)
        datan_2 = datan_2.replace('"', '', regex= True)
        if col == 'lx_route':
            datan_2[col] = datan_2[col].str.split(',').str[0]
        else: 
            datan_2[col] = datan_2[col].str.split('[').str[-1]  


    datan_3 = datan_2.set_index(timestamps2)
    datan_3 = datan_3[datan_3.lx_route != '/lx/channel/1/nextPattern']
    datan_3 = datan_3[datan_3.lx_route != '/lx/channel/2/nextPattern']
    datan_3 = datan_3[datan_3.lx_route != '/lx/channel/3/nextPattern']
    datan_3 = datan_3[datan_3.lx_route != '/lx/channel/4/nextPattern']
    datan_3 = datan_3[~datan_3.index.duplicated(keep='last')]

    training_data_pivoted = datan_3.pivot(columns = 'lx_route', values = 'data')
    td_p = training_data_pivoted
    
    placeholder = 0 
    j = 0 
    for col in td_p:
        i = 0
        placeholder = 0
        if 'activePattern' in col or 'hue' in col or 'speed' in col or 'enabled':
            for row in td_p[col]:
                #print row
                if row != None:
                    placeholder = row
                    #print row
                else:
                    #print placeholder /lx/output/enabled
                    #td_p.loc[col, row] = placeholder
                    #td_p.loc[i, col] = placeholder
                    td_p.iloc[i, td_p.columns.get_loc(col)] = placeholder

                    #print row , '=',  placeholder
                i += 1
            j += 1
     
    return td_p

In [ ]:
for index, value in enumerate(list(dataframe.index)):


In [ ]:
#token = 'BQDSWFeIkS-7Gj6H9Ou4mPKTQgDLrldfJLkyay9n-8xW6dA_hDHTD5LdvJ9_uoBFujU87zpHimZk4jGVCzFhaC5epl_Scsv_BHEKoG7qZ6Am1A-YdXkoMMv7bKJwzjVGYjwSp7JRUudkmJB8AJY42eXbpcYYL2LRRgE-'

if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace=False
    ranges = ['short_term', 'medium_term', 'long_term']
    #esults = sp.current_user_playing_track()

In [ ]:
if not os.path.exists(DATA_FOLDER):
	print 'data folder created'
	os.makedirs(DATA_FOLDER)

In [ ]:
reload(spotify_utils)

In [ ]:
reload(spotify_utils)

song name:  Long Train Runnin'
timestamp:  1517524316210
artist name:  The Doobie Brothers
artist id:  39T6qqI0jDtSWWioX8eGJz
song id:  2D86Et1GcZEP2XITcmZTs9

 starting spotify data with current playing song! 

[]
name:  Lazaretto
id:  3T76zPJz3tWL27FrjJe2ot
artist:  Jack White
duration ms:  219400
popularity:  62
played:  2018-02-01T22:29:34.892Z
type <type 'unicode'>
[]
name:  Do I Wanna Know?
id:  1dmeXS21t9WV6d3Fm5mxTi
artist:  CHVRCHES
duration ms:  258159
popularity:  48
played:  2018-02-01T22:28:35.912Z
type <type 'unicode'>
[]
name:  Painting a Hole
id:  5rhQo26ggW0kpAieLLUWaq
artist:  The Shins
duration ms:  284466
popularity:  52
played:  2018-02-01T22:27:34.457Z
type <type 'unicode'>
[]
name:  Name For You
id:  6qnaCx4wQQBqFd9XdQyWjC
artist:  The Shins
duration ms:  189493
popularity:  66
played:  2018-02-01T22:26:52.220Z
type <type 'unicode'>
[]
name:  Starman - 2012 Remastered Version
id:  0pQskrTITgmCMyr85tb9qq
artist:  David Bowie
duration ms:  254295
popularity:  74
pl

In [48]:
dd = 1517524288262.00
spotify_df_raw.index[0]- dd

27948.0

In [ ]:
dataframe_f = logger_to_training_data_all_channels(data_file)

dataframe_matched, X_test_matched = match_training_dataset(dataframe_f, time_test_scaled, X_test)

dataframe_matched['td_timestamp'] = time_test_scaled
dataframe_matched.reset_index()
dataframe_matched.set_index('td_timestamp')

#dataframe_toggle, toggle_timestamps, toggle_indexes, X_test_toggle = drop_runtime_toggle(dataframe_matched, X_test_matched)


#Y_color_toggle, Y_speed_toggle, Y_pattern_toggle = dataframe_to_labels(dataframe_toggle)



In [ ]:
dataframe_matched

In [ ]:


timestamp = []
songs = []
artists = []
popularities = []
ids = []
artist_ids = []
if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace=False
    ranges = ['short_term', 'medium_term', 'long_term']
    #esults = sp.current_user_playing_track()
    curr_results = sp.current_playback()
    #print curr_results
    #print 'current playback keys ', curr_results.keys()
    #print results['item']
    #print 'user playing track keys', results.keys()
    print 'song name: ', curr_results['item']['name']
    print 'timestamp: ', curr_results['timestamp']
    artist_info = curr_results['item']['artists']
    print 'artist name: ', artist_info[0]['name']
    print 'artist id: ', artist_info[0]['id']
    print '\n \n progress \n', curr_results['progress_ms']
    print '\n \n full song length', curr_results['item']['duration_ms']
    print '\n ddd \n' , curr_results['item']['id']
	#print curr_results['timestamp']
    #print 'type name: ', curr_results['item']['type']
    print 'song id: ', curr_results['item']['id']
    print '''




	'''
    recent_results = sp.current_user_recently_played(limit=5)
    print recent_results.keys()

    print recent_results['href']
    print recent_results['cursors']
    print recent_results['next']

    for i, item in enumerate(recent_results['items']):
    	print 'name: ', item['track']['name']
    	print 'id: ', item['track']['id']
    	artist_info = item['track']['artists']
    	print 'artist: ', artist_info[0]['name']
    	print 'duration ms: ', item['track']['duration_ms']

    	print 'popularity: ', item['track']['popularity']
        print '\n song length: ', item['track']['duration_ms']
        
        print 'played: ', item['played_at']
        print 'type', type(item['played_at'])
    	timestamp.append(item['played_at'])
        songs.append(item['track']['name'])
        ids.append(item['track']['id'])
        artists.append(artist_info[0]['name'])
        artist_ids.append(artist_info[0]['id'])
        popularities.append(item['track']['popularity'])

    
else:
    print "Can't get token for", username
 

In [ ]:
# just realized i should only create/save features at the end when i have all ids. then i
# only execute it when i need it!

def create_spotify_df(token, scope, get_genre=True, save_features=False):
    timestamp = []
    songs = []
    artists = []
    popularities = []
    ids = []
    genres = []
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace=False
        ranges = ['short_term', 'medium_term', 'long_term']
        #esults = sp.current_user_playing_track()
        curr_results = sp.current_playback()
        #print curr_results
        #print 'current playback keys ', curr_results.keys()
        #print results['item']
        #print 'user playing track keys', results.keys()
        print 'song name: ', curr_results['item']['name']
        print 'timestamp: ', curr_results['timestamp']
        artist_info = curr_results['item']['artists']
        print 'artist name: ', artist_info[0]['name']
        print 'artist id: ', artist_info[0]['id']



        #print curr_results['timestamp']
        #print 'type name: ', curr_results['item']['type']
        print 'song id: ', curr_results['item']['id']
        print '''




        '''
        recent_results = sp.current_user_recently_played(limit=5)
        print recent_results.keys()

        print recent_results['href']
        print recent_results['cursors']
        print recent_results['next']

        for i, item in enumerate(recent_results['items']):
            print 'name: ', item['track']['name']
            print 'id: ', item['track']['id']
            artist_info = item['track']['artists']
            print 'artist: ', artist_info[0]['name']
            print 'duration ms: ', item['track']['duration_ms']

            print 'popularity: ', item['track']['popularity']


            print 'played: ', item['played_at']
            print 'type', type(item['played_at'])
            timestamp.append(item['played_at'])
            songs.append(item['track']['name'])
            ids.append(item['track']['id'])
            artists.append(artist_info[0]['name'])
            popularities.append(item['track']['popularity'])
            if get_genre == True:
                genres.append(get_genre_from_track_id(sp, item['track']['id']))
                
        #get timestamps
        utc_timestamps = []
        for time in timestamp:

            utc_timestamps.append(calendar.timegm(dateutil.parser.parse(time).timetuple()))
        print utc_timestamps

        # create dataframe of basic data.
        spotify_data = {'timestamp': utc_timestamps, 'artist': artists, 'song': songs, 'id': ids, 'genre': genres, 'popularity': popularities}
        spotify_data_df = pd.DataFrame(spotify_data)
        spotify_data_df.set_index('timestamp')
        if save_features == True:
            try: 
                save_spotify_features(sp, ids)
                print 'features saved!'
            except:
                print 'unable to save features'

    return spotify_data_df

def save_spotify_features(sp, ids):
    # get audio features for each track and save as a JSON!
    features = sp.audio_features(ids)
    features_json = []
    for feature in features:
        #print feature
        print(json.dumps(feature, indent=4))
        print()
        features_json.append(json.dumps(feature, indent=4))
        with open('data.json', 'a') as outfile:
            json.dump(feature, outfile, indent=4)
        #analysis = sp._get(feature['analysis_url'])
        #print(json.dumps(analysis, indent=4))
        #print()
def get_album_id_from_track(sp, id):
    track = sp.track(id)
    #pprint.pprint(track)
    album_id = track['album']['id']
    
    print album_id
    return album_id
def get_genre_from_track_id(sp, id):
    track = sp.track(id)
    #pprint.pprint(track)
    album_id = track['album']['id']
    album = sp.album(str(album_id))
    pprint.pprint(album['genres'])
    return album['genres']
def get_audio_features_json(sp, dataframe, savefile):
    # get audio features for each track and save as a JSON!
    ids = spotify_data['id'].tolist()
    features = sp.audio_features(ids)
    features_json = []
    for feature in features:
        #print feature
        print(json.dumps(feature, indent=4))
        print()
        features_json.append(json.dumps(feature, indent=4))
        with open(savefile, 'a') as outfile:
            json.dump(feature, outfile, indent=4)
        # OPTIONAL song analysis (too much irrelavant data ATM.)    
        
        #analysis = sp._get(feature['analysis_url'])
        #print(json.dumps(analysis, indent=4))
        #print()
# to write- get all IDs from big pandas dataframe and then save spotify features
# save spotify data df as dataframe? or just sync it w training data.

In [ ]:
get_audio_features_json(sp, spotify_data_df, 'spotify_df_1_16.json')

In [ ]:
get_album_id_from_track(sp, '044qU7kwVZaDwzWnRsmVVL')

In [ ]:
spotify_data = create_spotify_df(token, scope)
spotify_data_df = pd.DataFrame(spotify_data).encode('utf-8')
spotify_data_df.set_index('timestamp')

In [ ]:
ids[0].encode('utf-8')

In [ ]:
spotify_data_df.to_csv('csvtest.csv')

In [ ]:
# get audio features for each track and save as a JSON!
features = sp.audio_features(ids)
features_json = []
for feature in features:
    #print feature
    print(json.dumps(feature, indent=4))
    print()
    features_json.append(json.dumps(feature, indent=4))
    with open('data.json', 'a') as outfile:
        json.dump(feature, outfile, indent=4)
    #analysis = sp._get(feature['analysis_url'])
    #print(json.dumps(analysis, indent=4))
    #print()

In [ ]:
#get timestamps
utc_timestamps = []
for time in timestamp:

    utc_timestamps.append(calendar.timegm(dateutil.parser.parse(time).timetuple()))
print utc_timestamps

In [ ]:
# create dataframe of basic data.
spotify_data = {'timestamp': utc_timestamps, 'artist': artists, 'song': songs, 'id': ids, 'popularity': popularities}


In [ ]:
parsed_t = dp.parse('2018-01-17T01:14:10.702Z')
t_in_seconds = parsed_t.strftime('%s')
print t_in_seconds

In [ ]:
calendar.timegm(dateutil.parser.parse('2018-01-17T01:14:10.702Z').timetuple())

In [ ]:
import datetime as datetime

In [ ]:
datetime.utcfromtimestamp(455051100)

In [ ]:
scope = 'user-read-playback-state user-read-recently-played'

token = util.prompt_for_user_token('aaronopp', scope, client_id='dbe2a20785304190b8e35d5d6644397b', client_secret='cc259d8378be48beaad9171a5afb19ba', redirect_uri='http://localhost:8888/callback')


start = time.time()
if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace=False
    ranges = ['short_term', 'medium_term', 'long_term']
    #esults = sp.current_user_playing_track()
    curr_results = sp.current_playback()
    #print curr_results
    #print 'current playback keys ', curr_results.keys()
    #print results['item']
    #print 'user playing track keys', results.keys()
    print 'song name: ', curr_results['item']['name']
    print 'timestamp: ', curr_results['timestamp']
    artist_info = curr_results['item']['artists']
    print 'artist name: ', artist_info[0]['name']
    print 'artist id: ', artist_info[0]['id']
	


	#print curr_results['timestamp']
    #print 'type name: ', curr_results['item']['type']
    print 'song id: ', curr_results['item']['id']
    
else:
    print "Can't get token for", username

print time.time() - start

In [5]:
sp = spotipy.Spotify(auth=token)
sp.trace=False

In [13]:
sp.current_playback()

{u'context': {u'external_urls': {u'spotify': u'https://open.spotify.com/artist/4yvcSjfu4PC0CYQyLy4wSq'},
  u'href': u'https://api.spotify.com/v1/artists/4yvcSjfu4PC0CYQyLy4wSq',
  u'type': u'artist',
  u'uri': u'spotify:artist:4yvcSjfu4PC0CYQyLy4wSq'},
 u'device': {u'id': u'8c8fb7719a082c39a867b40c1fbb7f571e25433e',
  u'is_active': True,
  u'is_restricted': False,
  u'name': u"Phil's Phone",
  u'type': u'Smartphone',
  u'volume_percent': None},
 u'is_playing': True,
 u'item': {u'album': {u'album_type': u'single',
   u'artists': [{u'external_urls': {u'spotify': u'https://open.spotify.com/artist/4yvcSjfu4PC0CYQyLy4wSq'},
     u'href': u'https://api.spotify.com/v1/artists/4yvcSjfu4PC0CYQyLy4wSq',
     u'id': u'4yvcSjfu4PC0CYQyLy4wSq',
     u'name': u'Glass Animals',
     u'type': u'artist',
     u'uri': u'spotify:artist:4yvcSjfu4PC0CYQyLy4wSq'}],
   u'available_markets': [u'AD',
    u'AR',
    u'AT',
    u'AU',
    u'BE',
    u'BG',
    u'BO',
    u'BR',
    u'CA',
    u'CH',
    u'CL',
 

In [12]:
sp.current_user()

{u'display_name': u'Philip Doueihi',
 u'external_urls': {u'spotify': u'https://open.spotify.com/user/1179511751'},
 u'followers': {u'href': None, u'total': 16},
 u'href': u'https://api.spotify.com/v1/users/1179511751',
 u'id': u'1179511751',
 u'images': [{u'height': None,
   u'url': u'https://scontent.xx.fbcdn.net/v/t1.0-1/p200x200/15697830_10154866811302298_5339394550715434321_n.jpg?_nc_cat=0&oh=b2a59ac18dbd574796148ecfd35541a8&oe=5B3EEE94',
   u'width': None}],
 u'type': u'user',
 u'uri': u'spotify:user:1179511751'}

In [ ]:
sp.